In [1]:
runtime = 24 # hrs
data_rate = 1000 # bps
total_bits = runtime * 60 * 60 * data_rate
# Make sure the number of bits is divisible by 8 perfectly so the file system doesn't melt itself
total_bits += 8 - total_bits % 8
print('File should be ' + '{:2.2f}'.format(total_bits / (8 * 1000 * 1000)) + ' MB')

import numpy as np

from numpy.random import PCG64
bg = PCG64(12345678903141592653589793)

# generate random data
random_data = (bg.random_raw(1024) % 2).astype(bool)

random_data, len(random_data)

File should be 10.80 MB


(array([False, False,  True, ...,  True,  True,  True]), 1024)

In [2]:
# pack the bits into a data array of bytes for writing
random_data_array = np.packbits(random_data)
random_data_array, len(random_data_array), len(random_data) / 8

(array([ 54, 235,  33, 185, 106, 143, 214, 148,  95, 229,  90,  27,  13,
        194, 210,  20,  73, 129,  98, 163, 163, 167, 107, 162, 249, 156,
        162,  59,  13,  51,  74,  48,  68,  83,  28, 237, 208, 170, 120,
        176, 200, 194,   8, 252, 172, 104, 255,  71, 106, 122, 187, 253,
          2,   4, 217, 107, 230,   2, 120, 155, 231, 211,  28, 219,  51,
        196, 213, 113,  95, 113, 250,  54, 187,   3, 150, 168, 140, 203,
         68, 121, 198, 117, 111, 249, 113, 136,  90, 182, 153, 241,  23,
         23, 142,  11,  58, 215, 163, 121, 117, 204, 216, 107,   4,  66,
        245, 139,  95, 168,   3,  84,  56,  93, 111, 152, 224,  22,  37,
         85,  88, 214, 179, 206, 195, 154, 121, 252, 182, 159], dtype=uint8),
 128,
 128.0)

In [3]:
def generate_next_kbit_of_data_as_byte_array(number_of_bytes=128, balance_slice_bits_length=32):
    if balance_slice_bits_length % 2 != 0:
        raise('Slice of bytes should be divisible by 2')

    def calculate_imbalance(array):
        return int(np.sum(array) - len(array) / 2)

    # Generate a random sequence of bits
    bit_array = (bg.random_raw(number_of_bytes * 8) % 2).astype(bool)

    # Flip random bits so the stream is DC balanced
    # Slice the bit stream into balance_slice_bits_length before balancing 
    # to guarantee that balance_slice_bits_length number of bits is balanced
    points = np.arange(0, len(bit_array), balance_slice_bits_length)
    groups = zip(points, np.append(points[1:], [len(bit_array)-1])) 

    # Flip the bits in groups until the group is balanced
    for begin, end in groups:
        bit_sequence = bit_array[begin:end]
        imbalance_value = calculate_imbalance(bit_sequence)
        while imbalance_value > 0:
            imbalance_value = calculate_imbalance(bit_sequence)
            ones = np.nonzero(bit_sequence)[0]
            bits_to_flip = bg.random_raw(imbalance_value) % len(ones)
            bit_sequence[ones[bits_to_flip]] = False
        while imbalance_value < 0:
            imbalance_value = calculate_imbalance(bit_sequence)
            zeros = np.nonzero(bit_sequence==0)[0]
            bits_to_flip = bg.random_raw(-1 * imbalance_value) % len(zeros)
            bit_sequence[zeros[bits_to_flip]] = True
    return np.packbits(bit_array)

total_bits_to_write = total_bits

with open("./tx_data", 'wb') as transmitted_binary:
    transmitted_binary.write(generate_next_kbit_of_data_as_byte_array())
    total_bits_to_write -= 128 * 8

    while total_bits_to_write > 0:
        if total_bits_to_write > 128 * 8:
            transmitted_binary.write(generate_next_kbit_of_data_as_byte_array())
            total_bits_to_write -= 128 * 8
        else:
            bytes_to_write = int(total_bits_to_write / 8)
            transmitted_binary.write(generate_next_kbit_of_data_as_byte_array(bytes_to_write))
            total_bits_to_write -= bytes_to_write * 8

if total_bits_to_write < 0:
    print('Error generating data')

In [4]:
number_of_bytes = 6
balance_slice_bits_length = 4 * 8

if balance_slice_bits_length % 2 != 0:
    raise('Slice of bytes should be divisible by 2')

def calculate_imbalance(array):
    return int(np.sum(array) - len(array) / 2)

# Generate a random sequence of bits
bit_array = (bg.random_raw(number_of_bytes * 8) % 2).astype(bool)

# Flip random bits so the stream is DC balanced
# Slice the bit stream into balance_slice_bits_length before balancing 
# to guarantee that balance_slice_bits_length number of bits is balanced
points = np.arange(0, len(bit_array), balance_slice_bits_length)
groups = zip(points, np.append(points[1:], [len(bit_array)-1])) 

# Flip the bits in groups until the group is balanced
for begin, end in groups:
    bit_sequence = bit_array[begin:end]
    imbalance_value = calculate_imbalance(bit_sequence)
    while imbalance_value > 0:
        imbalance_value = calculate_imbalance(bit_sequence)
        ones = np.nonzero(bit_sequence)[0]
        bits_to_flip = bg.random_raw(imbalance_value) % len(ones)
        bit_sequence[ones[bits_to_flip]] = False
    while imbalance_value < 0:
        imbalance_value = calculate_imbalance(bit_sequence)
        zeros = np.nonzero(bit_sequence==0)[0]
        bits_to_flip = bg.random_raw(-1 * imbalance_value) % len(zeros)
        bit_sequence[zeros[bits_to_flip]] = True

In [5]:
points

array([ 0, 32])

In [6]:
bit_array[len(bit_array)-1]

False

In [7]:
points = np.arange(0, len(bit_array), balance_slice_byte_length)
groups = zip(points, np.append(points[1:], [len(bit_array)-1])) 
for item in groups:
    print(item)

NameError: name 'balance_slice_byte_length' is not defined

In [ ]:
number_of_bytes = 128
# Generate a random sequence of bits
bit_array = (bg.random_raw(number_of_bytes * 8) % 2).astype(bool)
# Flip random bits so the stream is DC balanced
imbalance_value = np.sum(bit_array) - len(bit_array)
display(imbalance_value)
while imbalance_value > 0:
    imbalance_value = np.sum(bit_array) - len(bit_array)
    ones = np.nonzero(bit_array)[0]
    bits_to_flip = bg.random_raw(imbalance_value) % len(ones)
    bit_array[ones[bits_to_flip]] = False
while imbalance_value < 0:
    imbalance_value = np.sum(bit_array) - len(bit_array)
    zeros = np.nonzero(bit_array==0)[0]
    bits_to_flip = bg.random_raw(-1 * imbalance_value) % len(zeros)
    bit_array[zeros[bits_to_flip]] = True

np.sum(bit_array) - len(bit_array)

-529

0

In [ ]:
len(bits_to_flip)

0

In [ ]:
zeros = np.nonzero(bit_array==0)[0]
bg.random_raw(2), len(zeros)

(array([6931289548122753817, 5790276192407520242], dtype=uint64), 0)

In [ ]:
imbalance_value, bg.random_raw(-1 * imbalance_value) % len(zeros)

(0, array([], dtype=uint64))

In [ ]:
# Check that the written data makes sense
with open("tx_data", 'rb') as transmitted_binary:
    transmitted_array = np.asarray(bytearray(transmitted_binary.read()))

transmitted_bits = np.unpackbits(transmitted_array)

len(transmitted_bits), total_bits

(86401024, 86400008)